In [1]:
import brunoflow as bf
from jax import numpy as jnp, random

In [2]:
random_key_val = 42
num_embeddings = 5
embedding_dim = 3
padding_idx = 1
random_key = random.PRNGKey(random_key_val)


2022-12-19 13:23:17.410249: E external/org_tensorflow/tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error


In [3]:
weights = bf.Parameter(random.normal(key=random_key, shape=(num_embeddings, embedding_dim)))
weights.val = weights.val.at[padding_idx].set(0)
weights

node(name: None, val: [[-0.716899   -0.20865498 -2.5713923 ]
 [ 0.          0.          0.        ]
 [-0.8396519   0.3010434   0.1421263 ]
 [-1.7631724  -1.6755073   0.31390068]
 [ 0.5912831   0.5325395  -0.9133108 ]], grad: [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]])

In [4]:
emb_bf = bf.net.Embedding(num_embeddings=5, embedding_dim=3, padding_idx=1)
bf.matmul(emb_bf([2]), jnp.ones(shape=(3, 2))).backprop()
print(emb_bf.embeddings.grad)

[[0. 0. 0.]
 [0. 0. 0.]
 [2. 2. 2.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [5]:
import torch
emb = torch.nn.Embedding(num_embeddings=5, embedding_dim=3, padding_idx=1)
emb.weight

/home/kevin/miniconda3/envs/jax-hf/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Parameter containing:
tensor([[-0.7357, -1.2167, -1.3428],
        [ 0.0000,  0.0000,  0.0000],
        [-1.1554, -1.2417,  0.0507],
        [-0.0612,  0.1193,  0.2567],
        [-0.6271, -0.8092,  0.1092]], requires_grad=True)

In [6]:
# grad for a non-pad token
out = torch.matmul(emb(torch.tensor([2])), torch.ones(size=(3, 2), requires_grad=True))
out.backward(gradient=torch.ones_like(out))
emb.weight.grad

/home/kevin/miniconda3/envs/jax-hf/lib/python3.9/site-packages/torch/autograd/__init__.py:173: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /home/conda/feedstock_root/build_artifacts/pytorch-recipe_1664405705473/work/c10/cuda/CUDAFunctions.cpp:109.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


tensor([[0., 0., 0.],
        [0., 0., 0.],
        [2., 2., 2.],
        [0., 0., 0.],
        [0., 0., 0.]])

In [7]:
# grad for a pad token
out = torch.matmul(emb(torch.tensor([1])), torch.ones(size=(3, 2), requires_grad=True))
out.backward(gradient=torch.ones_like(out))
emb.weight.grad

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [2., 2., 2.],
        [0., 0., 0.],
        [0., 0., 0.]])